In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from pprint import pprint
import seaborn
import matplotlib
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from math import pi
from math import exp
import math
import sys
from sklearn.metrics import r2_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
eps = np.finfo(float).eps

In [2]:
def sigmoid(val):
    result = 1 / (1 + np.exp(-val+eps))
    return result

In [1]:
def gradientDescent(data, Q, target, alpha):
    for i in range(10000):
        arg = np.dot(data, Q)
        seg = sigmoid(arg)
        loss = (-target * np.log(seg + eps) - (1 - target) * np.log(1 - seg + eps)).mean()
        Q = Q - alpha * np.dot(data.T,(seg - target)) / len(target)
    return Q

In [3]:
def logisticRegression(data, target):
    return gradientDescent(data, np.zeros(data.shape[1]), target, 1)

In [5]:
def predict_one(test_row,models,unique_labels):
    result=[]
    for model in models:
        normalised = 0
        for i in range(len(model)):
            normalised += model[i] * data[i]
        result.append(sigmoid(normalised))
    maxIndex, index, it = -1, 0, 0
    for res in result:
        if res > maxIndex:
            index, maxIndex = it, res
        it += 1
    return unique_labels[index]

In [2]:
def predict(data, models, unique_labels):
    res = []
    for i in range(len(data)):
        result = predict_one(data[i], models, unique_labels)
        res.append(result)
    return res

### One Vs All

In [7]:
train_data = pd.read_csv('../input_data/Q3/wine-quality/data.csv',sep=';',header=None)

unique_labels = np.unique(np.array(train_data[11]))
models=[]
for x in unique_labels:
    train_data = pd.read_csv('../input_data/Q3/wine-quality/data.csv',sep=';',header=None)
    train_data[11] = np.where(train_data[11] != x, 0, train_data[11])
    train_data[11] = np.where(train_data[11] ==x, 1, train_data[11])
    X=train_data[[0,1,2,3,4,5,6,7,8,9,10]]
    Y=train_data[11]
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2,random_state=3)
    xm  = X_train.mean()
    xs = X_train.std()
    X_train=(X_train-X_train.mean())/X_train.std()
    X_test = (X_test-xm)/xs
    ones = np.ones([X_train.shape[0],1])
    X_train = np.concatenate((ones,X_train),axis=1)
    ones1 = np.ones([X_test.shape[0],1])
    X_test = np.concatenate((ones1,X_test),axis=1)
    models.append(logisticRegression(X_train,Y_train))

In [8]:
train_data = pd.read_csv('../input_data/Q3/wine-quality/data.csv',sep=';',header=None)
X=train_data[[0,1,2,3,4,5,6,7,8,9,10]]
Y=train_data[11]
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2,random_state=3)
xm  = X_train.mean()
xs = X_train.std()
X_train=(X_train-X_train.mean())/X_train.std()
X_test = (X_test-xm)/xs
ones1 = np.ones([X_test.shape[0],1])
X_test = np.concatenate((ones1,X_test),axis=1)
Y_pred = predict(X_test,models,unique_labels)
from sklearn.metrics import accuracy_score
print("Accuracy for One Vs All " ,accuracy_score(Y_test,Y_pred))

Accuracy for One Vs All  0.5204081632653061


#### Accuracy for One Vs All  52.04%

### One Vs One

In [196]:
train_data = pd.read_csv('../input_data/Q3/wine-quality/data.csv',sep=';',header=None)

unique_labels = np.unique(np.array(train_data[11]))
unique_labels.sort()
l = len(unique_labels)
models=[]
for i in range(l):
    for  j in range(i+1,l):
        train_data = pd.read_csv('../input_data/Q3/wine-quality/data.csv',sep=';',header=None)
        train_data = train_data[(train_data[11]==unique_labels[i]) | (train_data[11]==unique_labels[j])]
        train_data[11] = np.where(train_data[11] != unique_labels[i], 0, train_data[11])
        train_data[11] = np.where(train_data[11] ==unique_labels[i], 1, train_data[11])
        X=train_data[[0,1,2,3,4,5,6,7,8,9,10]]
        Y=train_data[11]

        X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2,random_state=3)
        xm  = X_train.mean()
        xs = X_train.std()
        X_train=(X_train-X_train.mean())/X_train.std()
        ones = np.ones([X_train.shape[0],1])
        X_train = np.concatenate((ones,X_train),axis=1)
        models.append(logisticRegression(X_train,Y_train))

[3 4 5 6 7 8 9]


### Prediction


In [202]:
def predict_onevsone(test_row,models,unique_labels):
    result=[]
    for model in models:
        normalised = 0
        for i in range(len(model)):
            normalised += model[i] * data[i]
        result.append(sigmoid(normalised)>0.5)
    l = len(unique_labels)
    res=[]
    x=0
    for i in range(l):
        for  j in range(i+1,l):
            if(result[x]==True):
                res.append(unique_labels[i])
            else:
                res.append(unique_labels[j])
            x+=1
    uni,c = np.unique(res,return_counts=True)
    real_answr = 0
    maxi=-1
    for i in range(len(c)):

        if(c[i]>maxi):
            maxi=c[i]
            real_answr = uni[i]
    return real_answr

In [203]:
def predict_ovo(rows,models,unique_labels):
    res=[]
    for i in range(len(rows)):
        res.append(predict_onevsone(rows[i],models,unique_labels))
    return res

In [204]:
train_data = pd.read_csv('../input_data/Q3/wine-quality/data.csv',sep=';',header=None)
X=train_data[[0,1,2,3,4,5,6,7,8,9,10]]
Y=train_data[11]
unique_labels = np.unique(Y)
unique_labels.sort()
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2,random_state=3)
xm  = X_train.mean()
xs = X_train.std()
X_train=(X_train-X_train.mean())/X_train.std()
X_test = (X_test-xm)/xs
ones1 = np.ones([X_test.shape[0],1])
X_test = np.concatenate((ones1,X_test),axis=1)
Y_pred = predict_ovo(X_test,models,unique_labels)
Y_pred
from sklearn.metrics import accuracy_score
print("Accuracy for One vs All",accuracy_score(Y_test,Y_pred))

7
0.5170068027210885


#### Accuracy for One Vs One 51.7 %